In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

"""Accuracy measures Logistic, not R2 as in the other models."""
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
"""__________________"""

from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
import datetime
from sklearn.preprocessing import StandardScaler
from scipy.stats import boxcox
import numpy as np

pd.set_option('display.max_columns', None)

import os

from statsmodels.formula.api import ols
import statsmodels.api as sm

import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

### Instructions
#### In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

##### Here is the list of steps to be followed (building a simple model without balancing the data):

* Import the required libraries and modules that you would need.
* Read that data into Python and call the dataframe churnData.
* Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. * * * * Convert this column into numeric type using pd.to_numeric function.
* Check for null values in the dataframe. Replace the null values.
* Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
* Scale the features either by using normalizer or a standard scaler.
* Split the data into a training set and a test set.
* Fit a logistic regression model on the training data.
* Check the accuracy on the test data.
Note: So far we have not balanced the data.

##### Managing imbalance in the dataset

* Check for the imbalance.
* Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
* Each time fit the model and see how the accuracy of the model is.

In [2]:
#IMPORT

churnData = pd.read_csv("Customer-Churn.csv")
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
#SELECT TYPES
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
#CHANGE TO NUMERIC
churnData["TotalCharges"] = pd.to_numeric(churnData["TotalCharges"], errors="coerce")

In [5]:
churnData.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [6]:
churnData["TotalCharges"].isna().sum()/len(churnData["TotalCharges"])

0.001561834445548772

In [7]:
#FILL NAs
churnData["TotalCharges"]=churnData["TotalCharges"].fillna(np.mean(churnData["TotalCharges"]))
churnData["TotalCharges"].isna().sum()/len(churnData["TotalCharges"])

0.0

In [8]:
#SELECT SPECIFIC COLUMNS
model=churnData[["tenure","SeniorCitizen", "MonthlyCharges","TotalCharges"]]

In [9]:
#SCALE DATA
scaled_data = StandardScaler().fit_transform(model) # Scale data 
model_scaled = pd.DataFrame(scaled_data, columns=model.columns) # Apply to dataframe

In [10]:
#CONCAT SCALED DATA WITH TARGET
data=pd.concat([model_scaled,churnData["Churn"]],axis=1)
#APPLY LABEL ENCODER IN TARGET
data["Churn"] = LabelEncoder().fit_transform(data["Churn"])

data.head()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
0,-1.277445,-0.439916,-1.160323,-0.994971,0
1,0.066327,-0.439916,-0.259629,-0.173876,0
2,-1.236724,-0.439916,-0.362660,-0.960399,1
3,0.514251,-0.439916,-0.746535,-0.195400,0
4,-1.236724,-0.439916,0.197365,-0.941193,1


In [11]:
#SPLIT DATA

X = data.drop("Churn",axis=1)
y = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
#APPLY LOGISTIC REGRESSION

lg = LogisticRegression()
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8076650106458482


In [13]:
#CHECK IMBALANCE!!!

data["Churn"].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

### DOWNSAMPLING

In [14]:
category_0=data[data["Churn"]==0]
category_1=data[data["Churn"]==1]

In [15]:
category_0=category_0.sample(frac=1)
category_0=category_0.sample(1869)

In [16]:
dwn=pd.concat([category_0,category_1],axis=0)
dwn["Churn"].value_counts()
dwn=dwn.sample(frac=1)
dwn

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
2297,0.880735,-0.439916,1.515170,1.593866,1
5798,-1.114563,-0.439916,0.679286,-0.798512,1
1663,0.758574,-0.439916,0.993366,1.116834,1
5547,-1.196004,-0.439916,0.820539,-0.901985,1
6452,-0.707359,-0.439916,0.764038,-0.459680,0
...,...,...,...,...,...
3382,1.613701,-0.439916,-1.303237,-0.183501,0
5049,-1.277445,-0.439916,-1.481050,-0.999232,1
3982,1.572981,-0.439916,-1.343121,-0.277592,0
4461,-0.137274,-0.439916,0.978410,0.143189,0


In [17]:
X = dwn.drop("Churn",axis=1)
y = dwn['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
#APPLY LOGISTIC REGRESSION

lg = LogisticRegression()
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)

accuracy_dw = accuracy_score(y_test, y_pred)
print(accuracy)

0.8076650106458482


### UPSAMPLING

In [19]:
category_0=data[data["Churn"]==0]
category_1=data[data["Churn"]==1]

In [20]:
category_1 = category_1.sample(frac=(5174/1869),replace=True)
up=pd.concat([category_0,category_1],axis=0)
up["Churn"].value_counts()
up=up.sample(frac=1)
up

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
3359,-0.463037,-0.439916,-1.489359,-0.824297,0
6806,0.310650,-0.439916,-1.517609,-0.663712,0
1833,-1.277445,-0.439916,-0.558752,-0.986980,0
4847,0.188489,2.273159,0.381825,0.246485,1
3410,-1.196004,-0.439916,-0.377617,-0.924834,1
...,...,...,...,...,...
3306,-1.196004,-0.439916,0.506459,-0.901609,0
6064,-1.277445,-0.439916,-0.646828,-0.988150,1
2395,-0.055834,2.273159,0.862084,0.243814,0
2231,-0.910961,-0.439916,0.174100,-0.681417,1


In [21]:
X = up.drop("Churn",axis=1)
y = up['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
#APPLY LOGISTIC REGRESSION

lg = LogisticRegression()
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)

accuracy_up= accuracy_score(y_test, y_pred)
print(accuracy)

0.8076650106458482


In [24]:
print("Accuracy with imbalance: ",accuracy)
print("Accuracy downsampling: ",accuracy_dw)
print("Accuracy upsampling: ",accuracy_up)

Accuracy with imbalance:  0.8076650106458482
Accuracy downsampling:  0.7232620320855615
Accuracy upsampling:  0.72512077294686
